In [1]:
import keras
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB


Using TensorFlow backend.


In [2]:
Train= pd.read_csv("../input/train.tsv", sep="\t")
Test = pd.read_csv("../input/test.tsv", sep="\t")



In [3]:
Train.shape

(156060, 4)

In [4]:
Train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [5]:
Train['Phrase'].str.len().mean()


40.217224144559786

In [6]:
Train['Phrase'].str.len().max()


283

In [7]:
Train['Sentiment'].value_counts()


2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [8]:
#Create df of full sentences
fullSent = Train.loc[Train.groupby('SentenceId')['PhraseId'].idxmin()]

#Change sentiment to increase readability
fullSent['sentiment_label'] = ''
Sentiment_Label = ['Negative', 'Somewhat Negative', 
                  'Neutral', 'Somewhat Positive', 'Positive']
for sent, label in enumerate(Sentiment_Label):
    fullSent.loc[Train.Sentiment == sent, 'sentiment_label'] = label
    
fullSent.head()

,PhraseId,SentenceId,Phrase,Sentiment,sentiment_label
0,1,1,A series of escapades demonstrating the adage ...,1,Somewhat Negative
63,64,2,"This quiet , introspective and entertaining in...",4,Positive
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1,Somewhat Negative
116,117,4,A positively thrilling combination of ethnogra...,3,Somewhat Positive
156,157,5,Aggressive self-glorification and a manipulati...,1,Somewhat Negative


In [9]:
#Add non-helpful stopwords to stopword list
Stopwords = list(ENGLISH_STOP_WORDS)
Stopwords.extend(['movie','movies','film','nt','rrb','lrb',
                      'make','work','like','story','time','little'])

#Create tfidf vectorizer object & fit to full sentence training data
tfidf_vectorizor = TfidfVectorizer(min_df=5, 
                             max_df=0.5,
                             analyzer='word',
                             strip_accents='unicode',
                             ngram_range=(1, 3),
                             sublinear_tf=True, 
                             smooth_idf=True,
                             use_idf=True,
                             stop_words=Stopwords)

tfidf_vectorizor.fit(list(fullSent['Phrase']))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=None,
                min_df=5, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['very', 'everyone', 'am', 'into', 'none', 'again',
                            'several', 'least', 'therefore', 'also', 'it',
                            'more', 'when', 'anything', 'up', 'becoming',
                            'another', 'many', 'namely', 'all', 'always', 'do',
                            'formerly', 'or', 'our', 'to', 'someone', 'whoever',
                            'whom', 'beside', ...],
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [10]:
BoW_vectorizer = CountVectorizer(strip_accents='unicode',
                                 stop_words=Stopwords,
                                 ngram_range=(1,3),
                                 analyzer='word',
                                 min_df=5,
                                 max_df=0.5)

BoW_vectorizer.fit(list(fullSent['Phrase']))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=5,
                ngram_range=(1, 3), preprocessor=None,
                stop_words=['very', 'everyone', 'am', 'into', 'none', 'again',
                            'several', 'least', 'therefore', 'also', 'it',
                            'more', 'when', 'anything', 'up', 'becoming',
                            'another', 'many', 'namely', 'all', 'always', 'do',
                            'formerly', 'or', 'our', 'to', 'someone', 'whoever',
                            'whom', 'beside', ...],
                strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [11]:
phrase = np.array(Train['Phrase'])
sentiment = np.array(Train['Sentiment'])
# build train and test datasets
phrase_train, phrase_test, sentiment_train, sentiment_test = train_test_split(phrase, 
                                                                              sentiment, 
                                                                              test_size=0.2, 
                                                                              random_state=4)

#TF-IDF
train_tfidfmatrix = tfidf_vectorizor.fit_transform(phrase_train)
test_tfidfmatrix = tfidf_vectorizor.transform(phrase_test)

#Vectorizer (Bag of Words Model)
train_simplevector = BoW_vectorizer.transform(phrase_train)
test_simplevector = BoW_vectorizer.transform(phrase_test)

In [12]:
def format_data(train, test, max_features, maxlen):
    
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.utils import to_categorical
    
    train = train.sample(frac=1).reset_index(drop=True)
    train['Phrase'] = train['Phrase'].apply(lambda x: x.lower())
    test['Phrase'] = test['Phrase'].apply(lambda x: x.lower())

    X = train['Phrase']
    test_X = test['Phrase']
    Y = to_categorical(train['Sentiment'].values)

    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(X))

    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=maxlen)
    test_X = tokenizer.texts_to_sequences(test_X)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    return X, Y, test_X

In [13]:
maxlen = 125
max_features = 10000

X, Y, test_X = format_data(Train, Test, max_features, maxlen)

In [14]:
X

array([[   0,    0,    0, ..., 1072,   63, 7286],
       [   0,    0,    0, ...,   20, 2287,   23],
       [   0,    0,    0, ..., 1969,    4, 2609],
       ...,
       [   0,    0,    0, ...,    4,   18,   27],
       [   0,    0,    0, ..., 6545,  278, 3161],
       [   0,    0,    0, ...,    0,    0, 8059]], dtype=int32)

In [15]:
Y

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [16]:
test_X

array([[   0,    0,    0, ...,  613, 1026,  393],
       [   0,    0,    0, ...,  613, 1026,  393],
       [   0,    0,    0, ...,    0,    0,   16],
       ...,
       [   0,    0,    0, ...,    2,  126, 5835],
       [   0,    0,    0, ...,    2,  126, 5835],
       [   0,    0,    0, ...,    0,  373, 2008]], dtype=int32)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=4)

In [18]:
from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
      
def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        precision = true_positives / (possible_positives + K.epsilon())
        return precision
      
        
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
from keras.layers import Input, Dense, Embedding, Flatten, Dropout, Activation, GlobalMaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras import optimizers


In [20]:
def baseline_cnn_model(fea_matrix, n_class, mode, compiler):
    model = Sequential()

# Input / Embdedding
    model.add(Embedding(max_features, 150, input_length=maxlen))

# CNN
    model.add(SpatialDropout1D(0.2))

    model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(128, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())
    model.add(Dense(5, activation='relu'))
    if n_class == 1 and mode == "cla":
        #model.add(Activation('sigmoid'))
        model.compile(optimizer=compiler, loss='binary_crossentropy',
                  metrics=['acc', f1_m,precision_m,recall_m])
    else:
        #model.add(Activation('softmax'))
        model.compile(optimizer=compiler, loss='categorical_crossentropy',
                  metrics=['acc', f1_m,precision_m,recall_m])
    return model
# Output layer
    #model.add(Dense(5, activation='sigmoid'))

In [21]:
epochs = 100
#batch_size = 32
lr = 1e-4
batch_size = 100
#num_epochs = 10
decay=1e-2
mode = "reg"
n_class = 3 #5

adm = optimizers.Adam(lr = lr, decay = decay)
sgd = optimizers.SGD(lr = lr, nesterov=True, momentum=0.7, decay=decay)
Nadam = optimizers.Nadam(lr = lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model = baseline_cnn_model(X_train, n_class, mode, Nadam)


In [22]:
#if n_class == 1 and mode == "cla":
#        model.add(Activation('sigmoid'))
#        model.compile(optimizer='adam', loss='binary_crossentropy',
#                  metrics=['acc', f1_m,precision_m,recall_m])
#else:
#        model.add(Activation('softmax'))
#        model.compile(optimizer='adam', loss='categorical_crossentropy',
#                  metrics=['acc', f1_m,precision_m,recall_m])

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 117045 samples, validate on 39015 samples
Epoch 1/100
117045/117045 [==============================] - 15s 128us/step - loss: 1.5002 - acc: 0.5134 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.2143 - val_acc: 0.5210 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/100
117045/117045 [==============================] - 11s 97us/step - loss: 1.1898 - acc: 0.5359 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.1340 - val_acc: 0.5563 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/100
117045/117045 [==============================] - 11s 97us/step - loss: 1.0473 - acc: 0.5943 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.0861 - val_acc: 0.5872 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/100
117045/117045 [==============================] - 11s 97us/step - loss

In [23]:
def print_metrics(accuracy, f1_score, precision, recall):
    print("Simple cnn model performance")
    print('Accuracy: ', np.round(accuracy, 4))
    print('Precision: ', np.round(precision, 4))
    print('Recall: ', np.round(recall, 4))
    print('F1 score: ', np.round(f1_score, 4))
    print('\n')
    
loss, accuracy, f1_score, precision, recall = model.evaluate(X, Y)
print_metrics(accuracy, f1_score, precision, recall)



156060/156060 [==============================] - 13s 82us/step
Simple cnn model performance
Accuracy:  0.8098
Precision:  0.3697
Recall:  0.3697
F1 score:  0.3697


